In [1]:
import numpy as np
from vqls_prototype.matrix_decomposition import (
    SymmetricDecomposition,
    MatrixDecomposition,
    PauliDecomposition,
)

In [2]:
A = np.random.rand(4,4)
A += A.T 


In [3]:
PD = PauliDecomposition(A)

In [7]:
PD.strings

['II', 'IX', 'IZ', 'XI', 'XX', 'XZ', 'YY', 'ZI', 'ZX', 'ZZ']

In [28]:
contraction = {
    'II': ('I',1), 
    'IX': ('X',1), 'IY': ('Y',1), 'IZ':('Z',1),
    'XI': ('X',1), 'YI': ('Y',-1), 'ZI': ('Z',1),
    'XX': ('I',1), 'YY': ('I',-1), 'ZZ': ('I',1),
    'XY': ('Z',1.0j), 'YX': ('Z',1.0j),
    'XZ': ('Y', -1.0j), 'ZX': ('Y', 1.0j),
    'YZ': ('X', -1.0j), 'ZY': ('X', -1.0j),
}

In [34]:
nstrings = len(PD.strings)
original_strings, new_strings, coeffs = [], [], []
for i1 in range(nstrings):
    for i2 in range(i1+1, nstrings):
        s1, s2 = PD.strings[i1], PD.strings[i2]
        original_strings.append((s1,s2))
        new_string = ''
        coeff = 1.
        for p1, p2 in zip(s1,s2):
            pauli, c = contraction[p1+p2]
            new_string += pauli 
            coeff *= c
        new_strings.append(new_string)
        coeffs.append(coeff)
len(np.unique(new_strings)), len(new_strings)

(15, 45)

In [37]:
from qiskit.quantum_info import Operator, Pauli 
n = len(new_strings)
for i in range(n):
    s1, s2 = original_strings[i]
    moriginal = Operator(Pauli(s1).conjugate() @ Pauli(s2)).data

    new_string = new_strings[i]
    mcontracted = Operator(Pauli(new_string))*coeffs[i]

    assert np.allclose(moriginal, mcontracted)

Operator([[0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]],
         input_dims=(2, 2), output_dims=(2, 2))
Operator([[0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]],
         input_dims=(2, 2), output_dims=(2, 2))
Operator([[0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]],
         input_dims=(2, 2), output_dims=(2, 2))
Operator([[0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]],
         input_dims=(2, 2), output_dims=(2, 2))
Operator([[0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0

In [27]:

Operator(Pauli('Z').conjugate() @ Pauli('Y')).data

array([[0.+0.j, 0.-1.j],
       [0.-1.j, 0.+0.j]])

In [25]:
1.0j * Operator(Pauli('Y')).data

array([[ 0.+0.j,  1.+0.j],
       [-1.+0.j,  0.+0.j]])